To use this project, you will need to download the datasets used in the code. To do so, you can run the following code in a Jupyter notebook cell.

This will open a window where you can select the datasets you want to download. Make sure to select the `wordnet`, `treebank`, `brown`, `nps_chat`, `conll2000`, `dependency_treebank`, `masc_tagged`, `multext_east`, `switchboard`, and `timit_tagged` datasets.

In [151]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Getting tag datasets

In [10]:
import nltk

word_universal_tags = {
    **dict(nltk.corpus.treebank_chunk.tagged_words(tagset="universal")),
    **dict(nltk.corpus.treebank_chunk.tagged_words(tagset="universal")),
    **dict(nltk.corpus.treebank.tagged_words(tagset='universal')),
    **dict(nltk.corpus.brown.tagged_words(tagset='universal')),
    **dict(nltk.corpus.nps_chat.tagged_words(tagset="universal")),
    **dict(nltk.corpus.conll2000.tagged_words(tagset="universal")),
    **dict(nltk.corpus.dependency_treebank.tagged_words()),
    **dict(nltk.corpus.masc_tagged.tagged_words(tagset="universal")),
    **dict(nltk.corpus.multext_east.tagged_words(tagset="universal")),
    **dict(nltk.corpus.switchboard.tagged_words(tagset="universal")),
    **dict(nltk.corpus.timit_tagged.tagged_words(tagset="universal"))
}

We also get a secondary synonyms dictionary, other that wordnet

In [11]:
from nltk.corpus import MWAPPDBCorpusReader
from nltk.corpus.util import LazyCorpusLoader
 
mwa_ppdb = LazyCorpusLoader(
    "mwa_ppdb",
    MWAPPDBCorpusReader,
    r"(?!README|\.).*",
    nltk_data_subdir="misc",
    encoding="utf8",
)

We try to build a first graph.

The graph will be saved in two files: `wordnet_{wn.get_version()}_synonims.tsv` and `wordnet_{wn.get_version()}_words.tsv`, where `wn.get_version()` is the version of the WordNet dataset you are using. The first file contains the edges of the graph, and the second file contains the nodes and their categories (determined by the nltk tagset).

In [15]:
from typing import List
from tqdm.auto import tqdm
from nltk.corpus import wordnet as wn

set_of_nodes = set()

edges = open(f"wordnet_{wn.get_version()}_synonims.tsv", "w")
nodes = open(f"wordnet_{wn.get_version()}_words.tsv", "w")

edges.write("source\tdestination\n")
nodes.write("node_name\tcategory\n")

entries = dict(mwa_ppdb.entries())

other_words = [
    e
    for s, d in entries.items()
    for e in (s, d)
]

for word in tqdm(
    list(wn.words()) + other_words,
    desc="Words",
    leave=False,
    dynamic_ncols=True,
):    
    if len(word.strip()) == 0:
        continue
    synonims = wn.synonyms(word)        
    terms_to_add_to_node_list = [word]
    synonims = [
        s
        for syns in synonims
        for s in syns
    ]

    if word in entries:
        synonims.append(entries[word])
        
    for synonim in synonims:
        if len(synonim.strip()) == 0:
            continue
        terms_to_add_to_node_list.append(synonim)
        edges.write(f"{word}\t{synonim}\n")
    
    category = None
    for term in terms_to_add_to_node_list:
        if term in word_universal_tags:
            category = word_universal_tags[term]
            break
    if category is None:
        category = "UNKNOWN"
    for term in terms_to_add_to_node_list:
        if term in set_of_nodes:
            continue
        if len(term.strip()) == 0:
            continue
        this_term_category = word_universal_tags.get(term, category)
        nodes.write(f"{term}\t{this_term_category}\n")
        set_of_nodes.add(term)

nodes.close()
edges.close()

Words:   0%|                                                                                                  …

We load the graph

In [16]:
from grape import Graph

graph = Graph.from_csv(
    node_path="wordnet_3.0_words.tsv",
    nodes_column="node_name",
    node_list_node_types_column="category",
    edge_path="wordnet_3.0_synonims.tsv",
    directed=False,
    name="WordNet Synonims"
)
graph.enable()
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>WordNet Synonims</h2><p class="paragraph">The undirected graph WordNet Synonims has 251.39K heterogeneous nodes and 413.71K edges. The graph contains 68.31K connected components (of which 25.53K are disconnected nodes), with the largest one containing 80.75K nodes and the smallest one containing a single node. The RAM requirements for the nodes and edges data structures are 26.88MB and 1.90MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 257, the mode degree is 1, the mean degree is 3.29 and the node degree median is 2.</p><p class="paragraph">The nodes with the highest degree centrality are take (degree 257 and node type VERB), get (degree 236 and node type VERB), pass (degree 223 and node type VERB), break (degree 221 and node type NOUN) and hold (degree 220 and node type VERB).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 27 node types, of which the 10 most common are UNKNOWN (153.57K nodes, 61.09%), NOUN (56.57K nodes, 22.50%), ADJ (15.05K nodes, 5.99%), VERB (13.70K nodes, 5.45%), NUM (4.31K nodes, 1.71%), ADV (2.92K nodes, 1.16%), X (941 nodes, 0.37%), NNP (819 nodes, 0.33%), NN (668 nodes, 0.27%) and CD (664 nodes, 0.26%). The RAM requirement for the node types data structure is 16.09MB.</p><h4 style="margin: 1em 0 0 0;">Unknown node types</h4><p class="paragraph">Nodes with unknown node types are nodes with a node type that was not provided during the creation of the graph, which may be desired as the output of a node-label holdout. The graph contains 11 nodes with unknown node type, which are avery (degree 1), 8931 (degree 1), &quot; (degree 1), % (degree 1), + (degree 1), : (degree 1), ~ (degree 3), &#x27;the (degree 1), - (degree 1) and .. (degree 2), plus other 1 nodes with unknown node types, making up 0.00 of the nodes.</p><h3 style="margin: 1em 0 0 0;">Topological Oddities</h3><p class="paragraph">A topological oddity is a set of nodes in the graph that <i>may be derived</i> by an error during the generation of the edge list of the graph and, depending on the task, could bias the results of topology-based models. In the following paragraph, we will describe the detected topological oddities.</p><h4 style="margin: 1em 0 0 0;">Singleton nodes</h4><p class="paragraph">A singleton node is a node disconnected from all other nodes. We have detected 25.53K singleton nodes in the graph, involving a total of 25.53K nodes (10.16%). The detected singleton nodes are:</p><ul class="small-columns"><li><p class="paragraph">a_cappella (node type UNKNOWN)</p></li>
<li><p class="paragraph">aboral (node type UNKNOWN)</p></li>
<li><p class="paragraph">abstractive (node type ADJ)</p></li>
<li><p class="paragraph">accelerando (node type UNKNOWN)</p></li>
<li><p class="paragraph">acetylic (node type UNKNOWN)</p></li>
<li><p class="paragraph">achondroplastic (node type UNKNOWN)</p></li>
<li><p class="paragraph">acid-forming (node typ

In [17]:
str(graph)

'<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>WordNet Synonims